In [ ]:
!pip install torch torchvision --quiet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
seed = 42

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

full_trainset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)

# 고정된 랜덤시드로 split
train_size = int(0.9 * len(full_trainset))
val_size = len(full_trainset) - train_size

trainset, validset = torch.utils.data.random_split(
    full_trainset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(seed)  # key point!
)

cuda


100%|██████████| 170M/170M [00:03<00:00, 43.4MB/s]


In [2]:
class MyNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MyNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, stride=4, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(64, 192, 5, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [3]:
def train(model, loader, optimizer, criterion):
    model.train()
    loss_sum, correct, total = 0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return loss_sum / len(loader), 100 * correct / total


def test(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100 * correct / total


In [4]:
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
validloader = DataLoader(validset, batch_size=128, shuffle=False)

In [15]:
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(512, 10)

for param in model.fc.parameters():
    param.requires_grad = True

for param in model.layer4.parameters():
    param.requires_grad = True

model = model.to(device)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

best_val_acc = 0.0

for epoch in range(20):
    loss, train_acc = train(model, trainloader, optimizer, criterion)
    val_acc = test(model, validloader)

    print(f"[Epoch {epoch+1}] Loss: {loss:.4f} | Train Acc: {train_acc:.2f}% | Valid Acc: {val_acc:.2f}%")

    # Validation 성능이 좋아질 때마다 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "my_model.pt")
        print(f"  --> Saved best model (Val Acc = {val_acc:.2f}%)")

print("Training finished.")
print("Best model saved to my_model.pt")



[Epoch 1] Loss: 0.4843 | Train Acc: 84.26% | Valid Acc: 89.30%
  --> Saved best model (Val Acc = 89.30%)
[Epoch 2] Loss: 0.1598 | Train Acc: 95.02% | Valid Acc: 90.46%
  --> Saved best model (Val Acc = 90.46%)
[Epoch 3] Loss: 0.0459 | Train Acc: 99.25% | Valid Acc: 91.04%
  --> Saved best model (Val Acc = 91.04%)
[Epoch 4] Loss: 0.0140 | Train Acc: 99.91% | Valid Acc: 91.68%
  --> Saved best model (Val Acc = 91.68%)
[Epoch 5] Loss: 0.0061 | Train Acc: 99.99% | Valid Acc: 91.72%
  --> Saved best model (Val Acc = 91.72%)


KeyboardInterrupt: 

In [17]:
testset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform,
)
testloader = DataLoader(testset, batch_size=128, shuffle=False)
# model = MyNet().to(device)
model.load_state_dict(torch.load("my_model.pt", map_location=device))
model.eval()
test_acc = test(model, testloader)
print(f"Final Test Accuracy: {test_acc:.2f}%")

Final Test Accuracy: 90.89%
